In [8]:
import torch
import torch_geometric as pyg
import h3

import numpy as np
from sklearn.neighbors import NearestNeighbors

from MarsDataset import MarsDataset

### Loading Data


In [9]:


msd = MarsDataset('data/test.zarr')

elat, elon = msd.get_lat(), msd.get_lon()
print(elat.shape, "\n", elon.shape)

ecoords = np.stack([elat, elon], axis= -1).reshape((-1, 2))
ecoords_rad = np.deg2rad(ecoords)



(36,) 
 (72,)


ValueError: all input arrays must have the same shape

In [ ]:
H_RESO = 1

h3_grid = sorted(set([h3.geo_to_h3(lat, lon, H_RESO) for lat, lon in ecoords]))

hcoords = np.array([h3.h3_to_geo(val) for val in h3_grid])
hcoords.shape


NameError: name 'ecoords' is not defined

### H3 Graph

In [ ]:
NUM_NEIGHBORS = 6

hcoords_rad = np.deg2rad(hcoords)
hneighbors = NearestNeighbors(n_neighbors=NUM_NEIGHBORS, metric='haversine', n_jobs=2)

hneighbors.fit(hcoords_rad)
hadjmat = hneighbors.kneighbors_graph(hcoords_rad, NUM_NEIGHBORS, mode='distance').tocoo()

hadjmat

# Preprocessing

In [ ]:
from sklearn.preprocessing import normalize

hadjmat_normed = normalize(hadjmat, norm='l1', axis=1)
hadjmat_normed.data = 1.0 - hadjmat_normed.data

hadjmat_normed

In [ ]:
h_to_h_has_isolated = pyg.utils.contains_isolated_nodes(pyg.utils.from_scipy_sparse_matrix(hadjmat_normed)[0])

assert not h_to_h_has_isolated, "Error"

In [ ]:
h2h_graph_key = ('h', 'to', 'h')

h2h_graph_data = {
    "edge_index": torch.from_numpy(np.stack([hadjmat.col, hadjmat.row], axis=0).astype(np.int64)),
    "edge_attr": torch.from_numpy(np.expand_dims(hadjmat_normed.data, axis=-1).astype(np.float32)),
    "hcoords_rad": torch.from_numpy(hcoords_rad.astype(np.float32)),
    "info": "h3_to_h3_graph",
}

# Era to H3 bipartite Graph

In [ ]:
NUM_MARTIAN_NEIGHBORS = 1
MARTIAN_RADIUS = 3389.5
H_CUTOFF = 800 # in km must be smaller I guess bc the Earth is bigger than Mars
RADIUS_H_TO_MARTIAN = H_CUTOFF / MARTIAN_RADIUS

In [ ]:
eneigh = NearestNeighbors(n_neighbors=NUM_MARTIAN_NEIGHBORS, metric='haversine', n_jobs=2)
eneigh.fit(ecoords_rad)

martian_to_h3_adjmat = eneigh.radius_neighbors_graph(hcoords_rad, RADIUS_H_TO_MARTIAN).tocoo()
martian_to_h3_adjmat

In [ ]:
#Norm

martian_to_h3_adjmat_normed = normalize(martian_to_h3_adjmat, norm='l1', axis=1)
martian_to_h3_adjmat_normed.data = 1.0 - martian_to_h3_adjmat_normed.data
#martian_to_h3_adjmat_normed

In [ ]:
martian_h_has_isolated = pyg.utils.contains_isolated_nodes(pyg.utils.from_scipy_sparse_matrix(martian_to_h3_adjmat_normed)[0])

assert not martian_h_has_isolated, "Error"

In [ ]:
m2h_graph_key  = {"martian", "to", "h"}

m2h_graph_data = {
    "edge_index": torch.from_numpy(np.stack([martian_to_h3_adjmat.col, martian_to_h3_adjmat.row], axis=0).astype(np.int64)),
    "edge_attr": torch.from_numpy(np.expand_dims(martian_to_h3_adjmat_normed.data, axis=-1).astype(np.float32)),
    "hcoords_rad": torch.from_numpy(hcoords_rad.astype(np.float32)),
    "ecoords_rad": torch.from_numpy(ecoords_rad.astype(np.float32)),
    "info": "martian_to_h3_graph",
}

In [ ]:
from torch_geometric.data import HeteroData

graphs = HeteroData(
    {
        m2h_graph_key: m2h_graph_data,
        h2h_graph_key: h2h_graph_data,
    }
)

In [ ]:
from plot_utils import plot_graph, plot_bipartite_graph

plot_graph("H -> H Graph",
           graphs[("h", "to", "h")],
           "hcoords_rad"
)

plot_bipartite_graph("Martian -> H Graph",
                    "blue", 
                    graphs[("martian", "to", "h")],
                    ("ecoords_rad", "hcoords_rad"),
                    graphs[("martian", "to", "h")]["ecoords_rad"],
                    graphs[("martian", "to", "h")]["hcoords_rad"]
)

In [ ]:
print("test")